In [1]:
# Importando bibliotecas necessárias.

import random
import requests
import scipy.stats as stats
import numpy as np
import pandas as pd

c:\Users\Pedro Antônio Silva\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# O Jogo da Forca
Primeiramente, é bom ter em mente o funcionamento do nosso jogo:
- Sabemos o tamanho da palavra;
- Sabemos onde a letra tentada está na palavra;
- Tentar uma letra e acertar não altera nossas vidas;
- Tentar adivinhar uma palavra errada termina o jogo;

In [2]:
class JogoDeForca:
    def __init__(self):
        import requests
        url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
        r = requests.get(url, allow_redirects=True)
        if r.status_code==200:
            self.content = str(r.content.decode()).split('\n')
    
    def novo_jogo(self, vidas=5):
        self.vidas = vidas
        self.palavra = random.choice([palavra.upper() for palavra in self.content])
        return len(self.palavra)

    def tentar_letra(self, letra):
        if self.vidas > 0:
            if letra in self.palavra:
                return [idx for idx in range(len(self.palavra)) if self.palavra[idx]==letra]
            else:
                self.vidas -= 1
                if self.vidas == 0:
                    print("Fim de jogo!")
                    return False
                else:
                    return []
        
    def tentar_palavra(self, palavra):
        if self.vidas > 0:
            if self.palavra.lower() == palavra.lower():
                return True
            else:
                self.vidas = 0
                return False

In [3]:
# Criando o dicionário já formatado para uso.

url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
r = requests.get(url, allow_redirects=True)

if r.status_code==200:
    content = str(r.content.decode()).split('\n')

dicio = []

for palavra in content:
    dicio.append(palavra.upper())


# O jogador

Com base nas informações que temos sobre o funcionamento do jogo, podemos definir algumas prioridades para nosso agente:

- Como não perdemos vidas por acertar uma letra, basta tentar a letra com maior probabilidade de acerto que estaremos jogando de forma ótima, exceto quando essa probabilidade diferir muito pouco das demais, para esse caso a escolha ótima seria dependente de predizer probabilidades futuras.
    - Não sabemos se esta exceção será relevante para o desempenho final do agente, então farei o agente priorizar escolhas sem contar com isso.
- O agente tentará palavras somente se for sua ultima vida ou se tiver sobrado apenas uma palavra nas possibilidades, já que tentar uma palavra precoçemente não gera informações novas, tem maior chance de erro e nos faz perder o jogo. 
- Sabendo o tamanho da palavra, podemos já eliminar possibilidades antes de fazer nossa primeira tentativa.
- Depois de tentarmos uma letra, podemos eliminar possibilidades que não a contenham no mesmo lugar.

In [4]:
data = []
class Jogador():
    def __init__(self):
        self.jogo = JogoDeForca()
        palavras_possíveis = []
        self.prev = []
        size = self.jogo.novo_jogo()
        self.s = size
        self.data = data
        for palavra in dicio:
            if len(palavra) == size :
                palavras_possíveis.append(palavra)
        self.palavras_possíveis = set(palavras_possíveis)
            
    def run(self):
        while self.jogo.vidas > 0:
            self.record()
            if (self.jogo.vidas > 1) & (len(self.palavras_possíveis) > 1):
                lp, p = self.maior_p()
                pos = self.jogo.tentar_letra(lp)
                new = []
                if pos != []:
                    for palavra in self.palavras_possíveis:
                        b = False
                        for i in pos:
                            if palavra[i] != lp:
                                b = True
                        if not b:
                            new.append(palavra)    
                else:
                    for palavra in self.palavras_possíveis:
                        if lp not in palavra:
                            new.append(palavra)
                self.prev.append(lp)
                self.palavras_possíveis = set(new)
            else:       
                guess = random.choice(list(self.palavras_possíveis))
                return self.jogo.tentar_palavra(guess)
    # extrai a letra com maior ocorrência e sua frequência.
    def maior_p(self):
        f = self.freq_calc()
        max = 0
        for k in f.keys():
            if (f[k] >= max) & (k not in self.prev):
                max = f[k]
                key = k
        return key,max
    # Calcula a frequência de ocorrência de letras em todas as palavras. (apenas uma é contada por palavra)
    def freq_calc(self):
        freq = {}
        n = len(self.palavras_possíveis)
        for palavra in self.palavras_possíveis:
            for letra in set(palavra):
                if letra not in freq:
                    freq[letra] = 1/n
                else:
                    freq[letra] += 1/n
        return freq
    
    def record(self):
        freq = self.freq_calc()
        data = stats.entropy(list(freq.values()), base=2)
        self.data.append(data)

In [6]:
# código feito para testar mil jogos com o agente em questão.
v = 0
l = 0
r = True
results = []
for _ in range(10):
    for _ in range(100):
        j1 = Jogador()
        r = j1.run()
        if r == True:
            v += 1
        else:
            l += 1
        results.append(v/(v+l))


In [ ]:
data = pd.DataFrame(results)
data.mean()

0 0


### Por que escolher a letra com maior probabilidade de ocorrência sem nos preocuparmos com entropia nos retorna um funcionamento tão bom?

Quando tentamos uma letra, com por exemplo 80% de ocorrência nas palavras possíveis, podemos obter um de dois resultados:

- Acertamos, então eliminamos menos possibilidades, porém não perdemos vidas, então aumentamos nossas chances de acerto sem perder nada com isso.

- Erramos, então reduzimos nossas possibilidades para o minimo possível naquela rodada, ao custo de uma vida.
